# Slice-wise 1D gaussian denoising

In [ ]:
# Mount Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
!pip install mrcfile
import mrcfile
from ipywidgets import *
import cv2
from google.colab.patches import cv2_imshow

## Configuration

In [ ]:
tomogram_name = "TS_01_nobeads"
sigma = 1.5 # Gaussian's sigma

## End configuration

In [ ]:
def compute_gaussian_kernel(sigma=1):
  number_of_coeffs = 3
  number_of_zeros = 0
  while number_of_zeros < 2 :
    delta = np.zeros(number_of_coeffs)
    delta[delta.size//2] = 1
    coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
    number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
    number_of_coeffs += 1
  return coeffs[1:-1]

kernel = compute_gaussian_kernel(sigma)
print(kernel)
print(np.sum(kernel))
plt.plot(kernel)
plt.show()

In [ ]:
!cp drive/Shareddrives/MissingWedge/tomograms/{tomogram_name}.rec .

In [ ]:
tomogram_MRC = mrcfile.open(f'{tomogram_name}.rec')
print(tomogram_MRC.data.dtype, tomogram_MRC.data.shape)

In [ ]:
tomogram = np.copy(tomogram_MRC.data)
#tomogram = np.copy(tomogram_MRC.data.astype(np.uint8))
#tomogram[100, 10:60, 10:60] = 255

In [ ]:
def filter_over_Z(tomogram, kernel):
  filtered_tomogram = np.zeros_like(tomogram).astype(np.float32)
  shape_of_tomogram = np.shape(tomogram)
  padded_tomogram = np.zeros(shape=(shape_of_tomogram[0] + kernel.size, shape_of_tomogram[1], shape_of_tomogram[2]))
  padded_tomogram[kernel.size//2:shape_of_tomogram[0] + kernel.size//2, ...] = tomogram
  Z_dim = tomogram.shape[0]
  for z in range(Z_dim):
    tmp_slice = np.zeros_like(tomogram[z, :, :]).astype(np.float32)
    for i in range(kernel.size):
      #tmp_slice += padded_tomogram[z + i - kernel.size//2, :, :] * kernel[i]
      tmp_slice += padded_tomogram[z + i, :, :] * kernel[i]
    #filtered_tomogram[(z - kernel.size//2) % Z_dim, :, :] = tmp_slice
    filtered_tomogram[z, :, :] = tmp_slice
    print('.', end='', flush=True)
  print()
  return filtered_tomogram

filtered_tomogram_Z = filter_over_Z(tomogram, kernel)

In [ ]:
def g(z=0):
  #cv2_imshow(cv2.normalize(tomogram[z, :, :].astype(np.uint8), None, 0, 255, cv2.NORM_MINMAX))
  cv2_imshow(cv2.normalize(tomogram[z, :, :], None, 0, 255, cv2.NORM_MINMAX))
  cv2_imshow(cv2.normalize(filtered_tomogram_Z[z, :, :], None, 0, 255, cv2.NORM_MINMAX))

interactive_plot = interactive(g, z=100)
interactive_plot

In [ ]:
with mrcfile.new(f'drive/Shareddrives/MissingWedge/{tomogram_name}_Z__sigma={sigma}.mrc', overwrite=True) as mrc:
  mrc.set_data(filtered_tomogram_Z.astype(np.float32))
  mrc.data

In [ ]:
def filter_over_Y(tomogram, kernel):
  filtered_tomogram = np.zeros_like(tomogram).astype(np.float32)
  shape_of_tomogram = np.shape(tomogram)
  padded_tomogram = np.zeros(shape=(shape_of_tomogram[0], shape_of_tomogram[1] + kernel.size, shape_of_tomogram[2]))
  padded_tomogram[:, kernel.size//2:shape_of_tomogram[1] + kernel.size//2, :] = tomogram
  Y_dim = tomogram.shape[1]
  for y in range(Y_dim):
    tmp_slice = np.zeros_like(tomogram[:, y, :]).astype(np.float32)
    for i in range(kernel.size):
      tmp_slice += padded_tomogram[:, y + i, :] * kernel[i]
    filtered_tomogram[:, y, :] = tmp_slice
    print('.', end='', flush=True)
  print()
  return filtered_tomogram

filtered_tomogram_ZY = filter_over_Y(filtered_tomogram_Z, kernel)

In [ ]:
def g(z=0):
  #cv2_imshow(cv2.normalize(tomogram[z, :, :].astype(np.uint8), None, 0, 255, cv2.NORM_MINMAX))
  cv2_imshow(cv2.normalize(tomogram[z, :, :], None, 0, 255, cv2.NORM_MINMAX))
  cv2_imshow(cv2.normalize(filtered_tomogram_ZY[z, :, :], None, 0, 255, cv2.NORM_MINMAX))

interactive_plot = interactive(g, z=100)
interactive_plot

In [ ]:
def filter_over_X(tomogram, kernel):
  filtered_tomogram = np.zeros_like(tomogram).astype(np.float32)
  shape_of_tomogram = np.shape(tomogram)
  padded_tomogram = np.zeros(shape=(shape_of_tomogram[0], shape_of_tomogram[1], shape_of_tomogram[2] + kernel.size))
  padded_tomogram[:, :, kernel.size//2:shape_of_tomogram[2] + kernel.size//2] = tomogram
  X_dim = tomogram.shape[2]
  for x in range(X_dim):
    tmp_slice = np.zeros_like(tomogram[:, :, x]).astype(np.float32)
    for i in range(kernel.size):
      tmp_slice += padded_tomogram[:, :, x + i] * kernel[i]
    filtered_tomogram[:, :, x] = tmp_slice
    print('.', end='', flush=True)
  print()
  return filtered_tomogram

filtered_tomogram_ZYX = filter_over_X(filtered_tomogram_ZY, kernel)

In [ ]:
def g(z=0):
  #cv2_imshow(cv2.normalize(tomogram[z, :, :].astype(np.uint8), None, 0, 255, cv2.NORM_MINMAX))
  cv2_imshow(cv2.normalize(tomogram[z, :, :], None, 0, 255, cv2.NORM_MINMAX))
  cv2_imshow(cv2.normalize(filtered_tomogram_ZYX[z, :, :], None, 0, 255, cv2.NORM_MINMAX))

interactive_plot = interactive(g, z=100)
interactive_plot

In [ ]:
with mrcfile.new(f'drive/Shareddrives/MissingWedge/{tomogram_name}__sigma={sigma}.mrc', overwrite=True) as mrc:
  mrc.set_data(filtered_tomogram_ZYX.astype(np.float32))
  mrc.data